In [1]:
from PIL import Image
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import numpy as np
import pandas as pd
import pickle as pkl
from sklearn.metrics import jaccard_score as IOU
from torchvision import models, transforms, io
from torch.utils.data import Dataset, DataLoader
import torch.nn.utils.prune as prune
import utils
import os
import time
import copy

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DATASET_PATH = 'ADE20K_2021_17_01/'
index_file = 'index_ade20k.pkl'
with open('{}/{}'.format(DATASET_PATH, index_file), 'rb') as f:
    index_ade20k = pkl.load(f)

objects_mat = index_ade20k['objectPresence']

# Find 150 most common object IDs and non-common object IDs
total_object_counts = np.sum(objects_mat, axis=1)
object_count_ids = np.argsort(total_object_counts)[::-1]
most_common_obj_ids = object_count_ids[:150]
irrelevant_obj_ids = object_count_ids[150:]
# Find image IDs where no irrelevant objects appear
irrelevant_obj_counts = np.sum(objects_mat[irrelevant_obj_ids], axis=0)
good_image_ids = np.argwhere(irrelevant_obj_counts == 0).flatten()
# Only common objects included
common_objects_mat = objects_mat[np.ix_(most_common_obj_ids, good_image_ids)]

# Maps {obj_ids: 0-149}
obj_id_map = {sorted(most_common_obj_ids)[idx]: idx + 1 for idx in range(150)}
obj_id_map[-1] = 0

# Pick out images to train/evaluate on
train_image_ids = []
test_image_ids = []
for i in good_image_ids:
    if 'training' in index_ade20k['folder'][i]:
        train_image_ids.append(i)
    elif 'validation' in index_ade20k['folder'][i]:
        test_image_ids.append(i)
    else:
        raise Exception('Invalid folder name.')

In [3]:
class SegmentationDataset(Dataset):
    def __init__(self, image_ids, root_dir, index_mat, transform=None, target_transform=None):
        """
        Args:
            image_ids (list): list of image IDs from ADE20K
            root_dir (string): Directory with all the images.
            index_mat (array): object array from index_ade20k.pkl
            transform (callable, optional): Optional transform to be applied
                on a sample.
            target_transform (callable, optional): Optional transform to be applied
                on a sample segmentation label.
        """
        self.image_ids = image_ids
        self.root_dir = root_dir
        self.index_ade20k = index_mat
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        img_name = os.path.join(self.root_dir, self.index_ade20k['folder'][image_id], 
                                self.index_ade20k['filename'][image_id])
        img_info = utils.loadAde20K(img_name)
        
        image = io.read_image(img_info['img_name']).float()
        class_mask = Image.fromarray(img_info['class_mask'], mode='I')
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(class_mask)
            
        sample = (image, label)

        return sample

In [4]:
input_size = 224
transform = transforms.Compose([
                transforms.Resize(input_size),
                transforms.CenterCrop(input_size),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

target_transform = transforms.Compose([
                transforms.Resize(input_size, interpolation=0),
                transforms.CenterCrop(input_size),
                transforms.ToTensor()
            ])

/usr/local/lib/python3.7/site-packages/torchvision/transforms/transforms.py:258: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [29]:
num_samples = 4
batch_size = 2
training_data = SegmentationDataset(train_image_ids[:num_samples], './', index_ade20k, transform=transform, target_transform=target_transform)
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=False)
testing_data = SegmentationDataset(test_image_ids[:num_samples], './', index_ade20k, transform=transform, target_transform=target_transform)
test_dataloader = DataLoader(testing_data, batch_size=batch_size, shuffle=False)

In [6]:
def encode_label(label_arr, obj_id_map):
    """
    Encode labels for evaluating loss
    label_arr (tensor): B x 1 x H x W
    obj_id_map: dictionary mapping label class IDs to new (0-150) range IDs
    """
    convert_label_ids = lambda i: obj_id_map[i-1]
    vect_convert_label_ids = np.vectorize(convert_label_ids)
    encoded_label = vect_convert_label_ids(label_arr.squeeze().cpu().numpy())
    
    return torch.tensor(encoded_label, dtype=torch.long)

def get_parameter_size(model):
    """
    Return model size in terms of parameters
    Each parameter is a float32 - 4 bytes
    """
    num_params = 0
    for p in model.parameters():
        num_params += torch.count_nonzero(p.flatten())
        
    total_bytes = num_params.item() * 4
    kb = total_bytes / 1000
    
    return {"# Params": num_params.item(),
            "Size in KB": kb}

In [7]:
def validate(model, test_dataloader):
    # testing pass
    test_start = time.time()
    running_accuracy = 0
    running_iou = 0
    with torch.no_grad():
        for images, labels in test_dataloader:
            images = images.to(device)
            output = model(images)['out']
            labels = encode_label(labels, obj_id_map).to(device)
            probs = torch.nn.functional.softmax(output, dim=1)
            preds = torch.argmax(probs, dim=1, keepdim=True).squeeze()
            num_correct = torch.sum((preds == labels).to(int)).item()
            iou = IOU(labels.detach().cpu().numpy().reshape(-1), preds.detach().cpu().numpy().reshape(-1), average='weighted')
            print('Testing accuracy: {}'.format(num_correct/(224*224*len(images))))
            print('Testing IOU score: {}'.format(iou))
            running_accuracy += num_correct/(224*224*len(images))
            running_iou += iou

    print("Testing time: {} seconds".format(time.time() - test_start))

    return {"Testing pixel accuracy": running_accuracy / len(test_dataloader),
            "Testing IOU accuracy": running_iou / len(test_dataloader)}

In [11]:
class ThresholdPruning(prune.BasePruningMethod):
    PRUNING_TYPE = "unstructured"

    def __init__(self, threshold):
        self.threshold = threshold

    def compute_mask(self, tensor, default_mask):
        return torch.abs(tensor) > self.threshold

# Knowledge Distillation

In [10]:
# Load teacher models
teacher_model_1 = models.segmentation.fcn_resnet50(pretrained=False, num_classes=151).to(device)
teacher_model_1.load_state_dict(torch.load('../scene_seg_models/fcn_resnet_50/epochs_20_weights.pkl', map_location=torch.device('cpu')))
teacher_model_2 = models.segmentation.deeplabv3_resnet50(pretrained=False, num_classes=151).to(device)
teacher_model_2.load_state_dict(torch.load('../scene_seg_models/deeplab_resnet_50/epochs_20_weights.pkl', map_location=torch.device('cpu')))

<All keys matched successfully>

In [30]:
model = copy.deepcopy(teacher_model_1) # fcn resnet 50

params_to_prune = [(module, "weight") for _, module in model.named_modules() if isinstance(module, torch.nn.Conv2d)]
prune.global_unstructured(params_to_prune, pruning_method=ThresholdPruning, threshold=0.025)
for _, module in model.named_modules():
    if isinstance(module, torch.nn.Conv2d):
        prune.remove(module, 'weight')

In [35]:
def train_model(model, teacher_models, train_dataloader, test_dataloader, obj_id_map, epochs=5, lr=0.1, momentum=0.8):

    train_start = time.time()

    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
    criterion_soft = torch.nn.CrossEntropyLoss()
    criterion_hard = torch.nn.CrossEntropyLoss()

    for i in range(epochs):
        epoch_start = time.time()

        # training pass
        running_loss = 0
        for images, labels in train_dataloader:
            images = images.to(device)
#             labels = labels.to(device)
            optimizer.zero_grad()
            
            shape, full_tensor = None, None
            for tm in teacher_models:
                soft_label = tm(images)['out']
                if shape is None:
                    shape = soft_label.size()
                    full_tensor = torch.zeros(shape)
                full_tensor = full_tensor.add(soft_label)
            soft_label_outputs = torch.div(full_tensor, len(teacher_models))
            label_probs = torch.nn.functional.softmax(soft_label_outputs, dim=1)
            soft_labels = torch.argmax(label_probs, dim=1, keepdim=True).squeeze()
                        
            output = model(images)['out']
            labels = encode_label(labels, obj_id_map).to(device)
            loss_soft = criterion_soft(output, soft_labels)
            loss_hard = criterion_hard(output, labels)
            loss = sum([loss_soft, loss_hard])
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            print('Batch finished...')
        print('Training loss: {}'.format(running_loss/len(train_dataloader)))
        print("Training time: {} seconds".format(time.time() - epoch_start))
#         torch.save(model, result_path+'/epochs_{}_model.pkl'.format(i+1))

        # testing pass
        test_start = time.time()
        running_accuracy = 0
        running_iou = 0
        with torch.no_grad():
            for images, labels in test_dataloader:
                images = images.to(device)
                output = model(images)['out']
                labels = encode_label(labels, obj_id_map).to(device)
                probs = torch.nn.functional.softmax(output, dim=1)
                preds = torch.argmax(probs, dim=1, keepdim=True).squeeze()
                num_correct = torch.sum((preds == labels).to(int)).item()
                iou = IOU(labels.detach().cpu().numpy().reshape(-1), preds.detach().cpu().numpy().reshape(-1), average='weighted')
                print('Testing accuracy: {}'.format(num_correct/(224*224*len(images))))
                print('Testing IOU score: {}'.format(iou))
                running_accuracy += num_correct/(224*224*len(images))
                running_iou += iou
        print("Testing time: {} seconds".format(time.time() - test_start))
        print('-----> Overall testing pixel accuracy: {}'.format(running_accuracy / len(test_dataloader)))
        print('-----> Overall testing IOU accuracy: {}'.format(running_iou / len(test_dataloader)))
        print("Epoch completed in {} seconds.".format(time.time() - epoch_start))


In [36]:
test_model = copy.deepcopy(model)
train_model(test_model, [teacher_model_1, teacher_model_2], train_dataloader, test_dataloader, obj_id_map)

Batch finished...
Batch finished...
Training loss: 11.110483646392822
Training time: 12.232490301132202 seconds
Testing accuracy: 0.31869818239795916
Testing IOU score: 0.12904685533053906
Testing accuracy: 0.11647002551020408
Testing IOU score: 0.042331315619023
Testing time: 2.373562812805176 seconds
-----> Overall testing pixel accuracy: 0.21758410395408162
-----> Overall testing IOU accuracy: 0.08568908547478103
Epoch completed in 14.60613489151001 seconds.
Batch finished...
Batch finished...
Training loss: 11.278098106384277
Training time: 12.46445631980896 seconds
Testing accuracy: 0.3963149713010204
Testing IOU score: 0.18301133116041066
Testing accuracy: 0.33655532525510207
Testing IOU score: 0.12749976503985794
Testing time: 2.3401072025299072 seconds
-----> Overall testing pixel accuracy: 0.36643514827806123
-----> Overall testing IOU accuracy: 0.1552555481001343
Epoch completed in 14.804692268371582 seconds.
Batch finished...
Batch finished...
Training loss: 9.67750692367553